This script creates box plots

In [3]:
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


Overview: Takes in a data directory and stores it in a dictionary that is more efficient to iterate through for creating box plots. Rather than iterating by subject id number in the CleanedData directory, it will iterate by gesture.

Dictionary will have 13 keys to represent the 13 different gestures. Each key will have a value of a list that contains two lists. The two lists will represent the freeform and instruction folder respectively. Within each of those two lists, they will contain 37 data files paths. 
Ex: {'PanLeft': [../session_F_PanSelect_sub1ID.csv, ../session_F_PanSelect_sub2ID.csv, ..][../session_I_PanSelect_sub1ID.csv, ../session_I_PanSelect_sub2ID.csv, ..]}

Cleaned data directory must have the following subfolders/files: ...\CleanedData\Sub#\Session_Sub#_Sess#\\file.csv
The names that are filled in are:
    - '#' in Sub# will be the subject's number
    - 'Session' in Session_Sub#_Sess# is either Freeform or Instructional, '#' in Sub# is the subject's number, and '#' in Sess# is the session number
    - 'file' in file.csv is the name of the file

In [4]:
cleaned_data_folder_path = "C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\CleanedData"

gesture_dict = {}

# Iterates through CleanedData directory to make a dictionary of data where the 13 keys are gesture types and its values 
# are two lists representing freeform and instructional. Each list will contain the path to the 37 csv data files.
for root, sub_folders, files in os.walk(cleaned_data_folder_path):
    for file in files:
        # Splits file name for gesture and session type identification
        file_split = file.split("_")
        file_gesture = file_split[3]
        file_session_type = file_split[2]

        # Ignores the thank you files
        if (file_session_type != 'F') and (file_session_type != 'I'):
            continue

        # If a gesture is not in the dictionary, make the gesture a new key with list values freeform and instructional
        if file_gesture not in gesture_dict:
            freeform_folder = []
            instructional_folder = []
            gesture_dict[file_gesture] = [freeform_folder, instructional_folder]

        if file_session_type == 'F':
            gesture_dict[file_gesture][0].append(os.path.join(root, file))
        else:
            gesture_dict[file_gesture][1].append(os.path.join(root, file))

{'BoxSelect': [['C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\CleanedData\\Sub1\\Freeform_Sub1_Sess1\\cleaned_session_F_BoxSelect_subjID_1_04-14-23_02-24-05.csv', 'C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\CleanedData\\Sub10\\Freeform_Sub10_Sess1\\cleaned_session_F_BoxSelect_subjID_10_05-10-23_02-37-47.csv', 'C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_BoxSelect_subjID_11_05-10-23_03-50-54.csv', 'C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess2\\cleaned_session_F_BoxSelect_subjID_11_06-05-23_03-22-09.csv', 'C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess3\\cleaned_session_F_BoxSelect_subjID_11_06-13-23_02-08-06.csv', 'C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\CleanedData\\Sub12\\Freeform_Sub12_Sess1\\cleaned_session_F_BoxSelect_subjID_12_05-10-23_04-39-38.csv', 'C:\\Users\\vrelax\\Desktop\\VRel

Overview: Methods to help find the length of a gesture's trial.

In [139]:
def read_file(file_path):
    """
    Reads a csv file into a dataframe.

    Parameters
    -----
    file_path : str
        path to a csv file

    Returns
    -----
    file : dataframe
        a dataframe of the input csv file
    """
    
    file = pd.read_csv(file_path)
    return file


def get_dlength(pt1X, pt1Y, pt1Z, pt2X, pt2Y, pt2Z):
    """
    Gets the distance between two points in 3d space.

    Parameters
    -----
    pt1X : float
        x-coordinate of point 1
    pt1Y : float
        y-coordinate of point 1
    pt1Z : float
        z-coordinate of point 1
    pt2X : float
        x-coordinate of point 2
    pt2Y : float
        y-coordinate of point 2
    pt2Z : float
        z-coordinate of point 2

    Returns
    -----
    d_length : float
        the distance between two points
    """

    xDistance = float(pt2X) - float(pt1X)
    yDistance = float(pt2Y) - float(pt1Y)
    zDistance = float(pt2Z) - float(pt1Z)
    d_length = math.sqrt((xDistance)**2 + (yDistance)**2 + (zDistance)**2)
    return d_length


def get_trial_hand_length(df, trial_num, hand):
    """
    Gets the length of a single trial.

    Parameters
    -----
    df : dataframe
        file dataframe to analyze
    trial_num : int
        trial number or gesture counter UI to analyze
    hand : char
        left or right controller to analyze the position/translation

    Returns
    -----
    d_length_sum : float
        the total length of the trial found by adding the distance between every two data points in that trial
    """

    testDF = df.copy()

    # Drops all of the rows where both not pressed & trials that don't have the same trial number
    testDF.drop(testDF[(testDF["trigger_pull_amount_left"] == 0) & (testDF["trigger_pull_amount_right"] == 0)].index, inplace=True)
    testDF.drop(testDF[(testDF['gesture_counter_UI']) != trial_num].index, inplace=True)
    testDF.reset_index(drop=True, inplace=True)

    d_length_sum = 0
    pt1X = ''
    pt1Y = ''
    pt1Z = ''        
    pt2X = ''
    pt2Y = ''
    pt2Z = ''
    for row in range(len(testDF.loc[:,(hand + "_controller_translation_x")])):
        if row == 0:
            pt1X = float((testDF.loc[:,(hand + "_controller_translation_x")]).iloc[row])
            pt1Y = float((testDF.loc[:,(hand + "_controller_translation_y")]).iloc[row])
            pt1Z = float((testDF.loc[:,(hand + "_controller_translation_z")]).iloc[row])
        else:
            if row != 1:
                pt1X = float(pt2X)
                pt1Y = float(pt2Y)
                pt1Z = float(pt2Z)
            pt2X = float((testDF.loc[:,(hand + "_controller_translation_x")]).iloc[row])
            pt2Y = float((testDF.loc[:,(hand + "_controller_translation_y")]).iloc[row])
            pt2Z = float((testDF.loc[:,(hand + "_controller_translation_z")]).iloc[row])
            if ((math.isnan(pt2X)) or (math.isnan(pt2Y)) or (math.isnan(pt2Z))):
                continue
            d_length_sum += get_dlength(pt1X, pt1Y, pt1Z, pt2X, pt2Y, pt2Z)
    return d_length_sum

Overview: Main method that outputs csv files that contains data points to create box plots.

Two types of csv files are outputted:
    - All gestures + sessions: Contains all of the gestures + sessions on one single file
        - Each column represents a gesture + session + controller. An example would be Zoom In Freeform (Left) or Pan Left Instructional (Right)
        - Will have file name all_gestures_lengths_data.csv
    - A single gesture + session: Contains a single gesture + session. 
        - There are two columns, the first representing the left controller and the second representing the right. 
        - A file example would be Zoom In Freeform or Pan Left Instructional with two columns, left and right.


In [138]:
all_gestures_df = pd.DataFrame()

gesture_lengths_path = 'C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\BoxPlots\\GesturesLengths'

# Iterate through dictionary and get the length of each gesture trial. Each column represents the gesture and session type as well as left/right hand.
# Each column contains 185 data points; 37 sessions * 5 trials for a specific gesture and session type = 185 data points
# Key: Gesture
# Value: 2 lists that represents freeform and instructional respectively; each list contains 37 data csv file paths


for gesture in gesture_dict:
    for session_type in gesture_dict[gesture]:
        # Gets session type for column name
        name_split = os.path.basename(session_type[0]).split("_")
        session = ""
        if name_split[2] == 'F':
            session = "Freeform"
        else:
            session = "Instructional"

        # Gets column name based on gesture and session type. EX) Pan Left Freeform
        # Get output path for a single gesture session
        column_name = ""
        output_path = gesture_lengths_path + "\\"
        for i in range(len(gesture)):
            if ((gesture[i].isupper() == True) and (i != 0)):
                column_name += " "
                output_path += "_"
            column_name += gesture[i]
            output_path += gesture[i].lower()
        column_name += " " + session
        output_path += "_" + session.lower() + '.csv'
        
        # List of data points for right and left hand for a specific gesture and trial
        data_points_left_list = []
        data_points_right_list = []
        # Iterates through each file and adds its 5 trial data its respective list
        for file_num in range(len(session_type)):
            file_df = pd.read_csv(session_type[file_num])
            for trial_num in range(5):
                data_points_right_list.append(get_trial_hand_length(file_df, trial_num+1, 'r'))
                data_points_left_list.append(get_trial_hand_length(file_df, trial_num+1, 'l'))

        # Adds two new columns to the total dataframe
        all_gestures_df[(column_name + " (Left)")] = data_points_left_list
        all_gestures_df[(column_name + " (Right)")] = data_points_right_list

        # Exports singles gesture session data file
        single_gesture_session_df = pd.DataFrame()
        single_gesture_session_df[(column_name + " (Left)")] = data_points_left_list
        single_gesture_session_df[(column_name + " (Right)")] = data_points_right_list
        if not os.path.exists(output_path):
            single_gesture_session_df.to_csv(output_path, header=True, index=False)



# Exports all gestures lengths data file
all_gestures_output_path = gesture_lengths_path + '\\all_gestures_lengths_data.csv'
if not os.path.exists(all_gestures_output_path):
    all_gestures_df.to_csv(all_gestures_output_path, header=True, index=False)